In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-aug-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2022/train.csv
/kaggle/input/tabular-playground-series-aug-2022/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/test.csv')
print('train', train.shape)
print('test', test.shape)
train.head()

train (26570, 26)
test (20775, 25)


,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


In [3]:
!pip install pycaret --ignore-installed llvmlite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 6.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 36.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 50.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.9/25.9 MB 49.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.2/435.2 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 79.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 47.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 94.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.6/636.6 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307

In [5]:
from pycaret.classification import *

In [6]:
train.columns

Index(['id', 'product_code', 'loading', 'attribute_0', 'attribute_1',
       'attribute_2', 'attribute_3', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_3', 'measurement_4', 'measurement_5',
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13',
       'measurement_14', 'measurement_15', 'measurement_16', 'measurement_17',
       'failure'],
      dtype='object')

In [7]:
train.describe(include = ['category', 'object'])

,product_code,attribute_0,attribute_1
count,26570,26570,26570
unique,5,2,3
top,C,material_7,material_8
freq,5765,21320,10865


In [9]:
clf_1 = setup(data = train, target = 'failure', session_id = 100, 
              normalize = True,
              transformation = True, 
              remove_multicollinearity = True,
              multicollinearity_threshold = 0.85,
              remove_outliers = True,
              handle_unknown_categorical = True,
              ignore_low_variance = True,
              imputation_type = 'iterative',
              numeric_iterative_imputer = 'lightgbm',
              categorical_iterative_imputer = 'lightgbm',
              categorical_imputation = 'mode',
              unknown_categorical_method = 'most_frequent',
              fix_imbalance = True,
              feature_interaction = True,
              interaction_threshold = 0.1,
              create_clusters = True,
              feature_selection = True,
              feature_selection_threshold = 0.7,
              log_experiment = True,
              data_split_stratify = True,
              fold = 5,
              n_jobs = -1,
              use_gpu = True,
              silent = True,
              categorical_features = ['attribute_0', 'attribute_1'],
              numeric_features = ['loading',
       'attribute_2', 'attribute_3',],
              ignore_features = ['id', 'product_code', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_3', 'measurement_4', 'measurement_5',
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13',
       'measurement_14', 'measurement_15', 'measurement_16', 'measurement_17' ]) # session_id equivalent to random_state

,Description,Value
0,session_id,100
1,Target,failure
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(26570, 26)"
5,Missing Values,True
6,Numeric Features,3
7,Categorical Features,2
8,Ordinal Features,False
9,High Cardinality Features,False


2022/08/13 17:31:25 INFO mlflow.tracking.fluent: Experiment with name 'clf-default-name' does not exist. Creating a new experiment.


PicklingError: Can't pickle <class 'pyod.models.pca.PCA'>: it's not found as pyod.models.pca.PCA

In [ ]:
#types of models available in the library
models()

In [10]:
blender_soft = blend_models(compare_models(include = ['lr', 'nb', 'lightgbm', 'ridge', 'ada', 'xgboost'], n_select = 3, 
                                           sort = 'AUC'),
                            method = 'soft', weights = [0.5,0.3,0.2],
                            choose_better = True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6324,0.5747,0.4128,0.2654,0.3231,0.0869,0.0909
1,0.6299,0.5766,0.4154,0.2642,0.3230,0.0854,0.0895
2,0.6395,0.5829,0.4341,0.2774,0.3385,0.1070,0.1120
3,0.5893,0.5607,0.4653,0.2495,0.3248,0.0669,0.0736
4,0.6394,0.5989,0.4847,0.2910,0.3636,0.1334,0.1419
Mean,0.6261,0.5788,0.4425,0.2695,0.3346,0.0959,0.1016
Std,0.0188,0.0124,0.0282,0.0139,0.0156,0.0226,0.0236


In [12]:
blender_soft

GaussianNB(priors=None, var_smoothing=1e-09)

In [11]:
tuned_blend = tune_model(blender_soft)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6474,0.5707,0.3702,0.2645,0.3085,0.0807,0.0827
1,0.6488,0.5775,0.3728,0.2667,0.3109,0.0840,0.0861
2,0.6641,0.5894,0.4221,0.2963,0.3482,0.1312,0.1349
3,0.6128,0.5631,0.4147,0.2508,0.3126,0.0653,0.0693
4,0.6502,0.5997,0.4434,0.2893,0.3502,0.1251,0.1304
Mean,0.6447,0.5801,0.4046,0.2735,0.3261,0.0972,0.1007
Std,0.0170,0.0131,0.0287,0.0168,0.0189,0.0261,0.0268


In [13]:
print(tuned_blend)

GaussianNB(priors=None, var_smoothing=0.009)


In [17]:
evaluate_model(tuned_blend)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [18]:
# predictions on validation set
predict_model(tuned_blend)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.6422,0.5797,0.4165,0.2748,0.3311,0.1008,0.1048


,data_cluster_16,data_cluster_0,data_cluster_10,attribute_1_material_5,data_cluster_7,data_cluster_8,attribute_2,data_cluster_3,data_cluster_19,data_cluster_14,...,loading_multiply_data_cluster_7,loading_multiply_attribute_2,loading_multiply_data_cluster_5,loading_multiply_data_cluster_19,attribute_1_material_8_multiply_loading,loading_multiply_attribute_1_material_6,loading_multiply_data_cluster_3,failure,Label,Score
0,0.0,1.0,0.0,0.0,0.0,0.0,1.388293,0.0,0.0,0.0,...,0.000000,0.489661,0.000000,0.0,0.352707,0.0,0.0,0,1,0.8597
1,0.0,0.0,0.0,1.0,0.0,0.0,-0.392943,0.0,0.0,0.0,...,-0.000000,0.171700,-0.000000,-0.0,-0.000000,-0.0,-0.0,0,0,0.9542
2,0.0,0.0,0.0,1.0,0.0,0.0,-0.392943,0.0,0.0,0.0,...,-0.000000,0.476392,-0.000000,-0.0,-0.000000,-0.0,-0.0,0,0,0.9986
3,0.0,0.0,0.0,1.0,0.0,0.0,0.925445,0.0,0.0,0.0,...,-0.000000,-0.462256,-0.499496,-0.0,-0.000000,-0.0,-0.0,0,0,0.9992
4,0.0,0.0,0.0,0.0,1.0,0.0,-1.352000,0.0,0.0,0.0,...,-1.120189,1.514496,-0.000000,-0.0,-1.120189,-0.0,-0.0,0,0,0.9985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7967,0.0,0.0,1.0,0.0,0.0,0.0,1.388293,0.0,0.0,0.0,...,0.000000,2.590795,0.000000,0.0,1.866173,0.0,0.0,0,1,1.0000
7968,0.0,0.0,0.0,1.0,0.0,0.0,-0.392943,0.0,0.0,0.0,...,-0.000000,0.066950,-0.000000,-0.0,-0.000000,-0.0,-0.0,0,0,0.9433
7969,0.0,0.0,0.0,0.0,0.0,0.0,-1.352000,0.0,0.0,0.0,...,0.000000,-0.380955,0.000000,0.0,0.281772,0.0,0.0,0,0,0.5730
7970,0.0,1.0,0.0,0.0,0.0,0.0,1.388293,0.0,0.0,0.0,...,0.000000,0.597873,0.000000,0.0,0.430653,0.0,0.0,0,1,0.9191


In [20]:
final_blend =finalize_model(tuned_blend)

In [21]:
print(final_blend)

GaussianNB(priors=None, var_smoothing=0.009)


In [22]:
predictions = predict_model(final_blend, data = test)
predictions.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,Label,Score
0,26570,F,119.57,material_5,material_6,6,4,6,9,6,...,15.909,18.070,13.772,13.659,16.825,13.742,17.710,634.612,0,0.9505
1,26571,F,113.51,material_5,material_6,6,4,11,8,0,...,13.998,NaN,12.473,17.468,16.708,14.776,14.102,537.037,0,0.9613
2,26572,F,112.16,material_5,material_6,6,4,8,12,4,...,17.046,18.086,10.907,13.363,15.737,17.065,16.021,658.995,0,0.9634
3,26573,F,112.72,material_5,material_6,6,4,8,11,10,...,18.165,16.163,10.933,15.501,15.667,12.620,16.111,594.301,0,0.9626
4,26574,F,208.00,material_5,material_6,6,4,14,16,8,...,14.578,17.849,11.941,16.070,16.183,13.324,17.150,801.044,1,1.0000


In [23]:
submission = pd.DataFrame()
submission['id'] = test['id']
submission['failure'] = predictions['Label']
submission.to_csv("submission.csv", index = False)

In [26]:
save_model(final_blend, 'Naive_Bayes_blend_13sep2022_2310')

PicklingError: Can't pickle <class 'pyod.models.pca.PCA'>: it's not found as pyod.models.pca.PCA